In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicki/ng run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stare-dataset/im0296.ppm
/kaggle/input/stare-dataset/im0372.ppm
/kaggle/input/stare-dataset/im0301.ppm
/kaggle/input/stare-dataset/im0209.ppm
/kaggle/input/stare-dataset/im0106.ppm
/kaggle/input/stare-dataset/im0089.ppm
/kaggle/input/stare-dataset/im0232.ppm
/kaggle/input/stare-dataset/im0069.ppm
/kaggle/input/stare-dataset/im0342.ppm
/kaggle/input/stare-dataset/im0395.ppm
/kaggle/input/stare-dataset/im0302.ppm
/kaggle/input/stare-dataset/im0087.ppm
/kaggle/input/stare-dataset/im0227.ppm
/kaggle/input/stare-dataset/im0350.ppm
/kaggle/input/stare-dataset/im0074.ppm
/kaggle/input/stare-dataset/im0250.ppm
/kaggle/input/stare-dataset/im0333.ppm
/kaggle/input/stare-dataset/im0288.ppm
/kaggle/input/stare-dataset/im0192.ppm
/kaggle/input/stare-dataset/im0399.ppm
/kaggle/input/stare-dataset/im0247.ppm
/kaggle/input/stare-dataset/im0007.ppm
/kaggle/input/stare-dataset/im0240.ppm
/kaggle/input/stare-dataset/im0071.ppm
/kaggle/input/stare-dataset/im0343.ppm
/kaggle/input/stare-datas

In [2]:
#pip install LPIPS

In [3]:
#pip install torch-fidelity

In [4]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils as vutils
from torchvision.models import inception_v3,Inception_V3_Weights
from torchvision.transforms import Resize
from torch.nn.functional import adaptive_avg_pool2d
import torch.nn.utils.spectral_norm as spectral_norm
from scipy import linalg
from scipy.linalg import sqrtm
from PIL import ImageOps
from PIL import ImageEnhance
from torchmetrics.image.kid import KernelInceptionDistance

In [5]:
class STAREPreprocessor:
    def __init__(self, input_dir, output_base, resize_size=(256, 256)):
        """
        Initialize the STARE dataset preprocessor
        
        Args:
            input_dir: Directory containing original STARE .ppm images
            output_base: Base directory for processed outputs
            resize_size: Target size for resized images (default: 256x256)
        """
        self.input_dir = input_dir
        self.output_base = output_base
        self.resize_size = resize_size
        
        # Output folders
        self.resize_dir = os.path.join(output_base, "resized")
        self.augment_dir = os.path.join(output_base, "augmented")
        self.enhance_dir = os.path.join(output_base, "enhanced")
        
        # Create directories
        os.makedirs(self.resize_dir, exist_ok=True)
        os.makedirs(self.augment_dir, exist_ok=True)
        os.makedirs(self.enhance_dir, exist_ok=True)
        
        # Data augmentation
        self.augmentations = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(20),
            transforms.RandomVerticalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2)
        ])
    
    def resize_with_padding(self, img, target_size, fill_color=(0, 0, 0)):
        """
        Resize image while maintaining aspect ratio and adding padding if needed
        
        Args:
            img: PIL Image
            target_size: Desired size (width, height)
            fill_color: Color to use for padding
            
        Returns:
            Resized and padded PIL Image
        """
        img.thumbnail(target_size, Image.LANCZOS)
        delta_w = target_size[0] - img.size[0]
        delta_h = target_size[1] - img.size[1]
        padding = (
            delta_w // 2, delta_h // 2,
            delta_w - (delta_w // 2), delta_h - (delta_h // 2)
        )
        return ImageOps.expand(img, padding, fill_color)
    
    def enhance_image(self, img):
        """
        Enhance image with contrast, sharpness, and color adjustments
        
        Args:
            img: PIL Image
            
        Returns:
            Enhanced PIL Image
        """
        img = ImageEnhance.Contrast(img).enhance(1.5)
        img = ImageEnhance.Sharpness(img).enhance(2.0)
        img = ImageEnhance.Color(img).enhance(1.3)
        return img
    
    def process_images(self, num_augmentations=2, save_enhanced=True, save_resized=True):
        """
        Process all images in the input directory
        
        Args:
            num_augmentations: Number of augmented versions to create per image
            save_enhanced: Whether to save enhanced versions
            save_resized: Whether to save resized versions
        """
        # Get all PPM files
        image_files = [f for f in os.listdir(self.input_dir) if f.endswith(".ppm")]
        
        # Process each image
        for filename in tqdm(image_files, desc="Processing images"):
            img_path = os.path.join(self.input_dir, filename)
            image = Image.open(img_path).convert("RGB")
            
            # Get base filename without extension
            base_filename = os.path.splitext(filename)[0]
            
            # Resize with padding to maintain aspect ratio
            resized = self.resize_with_padding(image, self.resize_size)
            
            # Save resized image
            if save_resized:
                resized.save(f"{self.resize_dir}/{base_filename}.png")
            
            # Save original in augmented folder
            resized.save(f"{self.augment_dir}/{base_filename}.png")
            
            # Create and save augmented versions
            for i in range(num_augmentations):
                augmented = self.augmentations(resized)
                augmented.save(f"{self.augment_dir}/{base_filename}_aug{i}.png")
            
            # Create and save enhanced version
            if save_enhanced:
                enhanced = self.enhance_image(resized)
                enhanced.save(f"{self.enhance_dir}/{base_filename}_enhanced.png")
                
                # Also create augmented enhanced versions
                for i in range(num_augmentations):
                    aug_enhanced = self.augmentations(enhanced)
                    aug_enhanced.save(f"{self.enhance_dir}/{base_filename}_enhanced_aug{i}.png")

In [6]:
class STAREDataset(Dataset):
    def __init__(self, data_dir, transform=None, use_augmented=True, use_enhanced=True):
        """
        PyTorch Dataset for processed STARE images
        
        Args:
            data_dir: Base directory containing processed images
            transform: Additional transforms to apply
            use_augmented: Whether to include augmented images
            use_enhanced: Whether to include enhanced images
        """
        self.data_dir = data_dir
        self.transform = transform
        
        # Collect image paths
        self.image_paths = []
        
        # Add resized images
        resize_dir = os.path.join(data_dir, "resized")
        if os.path.exists(resize_dir):
            self.image_paths.extend([
                os.path.join(resize_dir, f) for f in os.listdir(resize_dir)
                if f.endswith(('.png', '.jpg'))
            ])
        
        # Add augmented images
        if use_augmented:
            augment_dir = os.path.join(data_dir, "augmented")
            if os.path.exists(augment_dir):
                self.image_paths.extend([
                    os.path.join(augment_dir, f) for f in os.listdir(augment_dir)
                    if f.endswith(('.png', '.jpg'))
                ])
        
        # Add enhanced images
        if use_enhanced:
            enhance_dir = os.path.join(data_dir, "enhanced")
            if os.path.exists(enhance_dir):
                self.image_paths.extend([
                    os.path.join(enhance_dir, f) for f in os.listdir(enhance_dir)
                    if f.endswith(('.png', '.jpg'))
                ])
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image


In [7]:
input_dir = "/kaggle/input/stare-dataset"
output_base = "/kaggle/working/stare_prepared"
# Create and run preprocessor
preprocessor = STAREPreprocessor(input_dir, output_base, resize_size=(256, 256))
preprocessor.process_images(num_augmentations=3, save_enhanced=True)
    
# Create dataset with normalization transform
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
dataset = STAREDataset(output_base, transform=transform)
# Create dataloader
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

Processing images: 100%|██████████| 397/397 [01:40<00:00,  3.94it/s]


In [8]:
latent_dim = 100 

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DCGANGenerator(nn.Module):
    def __init__(self, z_dim=100, channels=3, features_g=64):
        """
        DCGAN Generator implementation
        
        Args:
            z_dim: Size of the noise vector
            channels: Number of output channels (3 for RGB)
            features_g: Size factor for generator (higher = more capacity)
        """
        super(DCGANGenerator, self).__init__()
        self.z_dim = z_dim
        
        # Starting with 1x1 projection from noise
        self.project = nn.Linear(z_dim, features_g * 8 * 4 * 4)
        
        # Main convolutional blocks for upsampling
        self.main = nn.Sequential(
            # Block 1: 4x4 -> 8x8
            nn.ConvTranspose2d(features_g * 8, features_g * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),
            
            # Block 2: 8x8 -> 16x16
            nn.ConvTranspose2d(features_g * 4, features_g * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),
            
            # Block 3: 16x16 -> 32x32
            nn.ConvTranspose2d(features_g * 2, features_g, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),
            
            # Block 4: 32x32 -> 64x64
            nn.ConvTranspose2d(features_g, features_g // 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g // 2),
            nn.ReLU(True),
            
            # Block 5: 64x64 -> 128x128
            nn.ConvTranspose2d(features_g // 2, features_g // 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g // 4),
            nn.ReLU(True),
            
            # Output layer: 128x128 -> 256x256
            nn.ConvTranspose2d(features_g // 4, channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()  # Output range: [-1, 1]
        )
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)
        elif classname.find('Linear') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias.data, 0)
    
    def forward(self, z):
        # Project and reshape
        x = self.project(z)
        x = x.view(-1, 8 * self.z_dim // 25, 4, 4)  # Shape becomes (batch_size, features_g*8, 4, 4)
        # Pass through main convolutional blocks
        return self.main(x)

In [10]:
class DCGANDiscriminator(nn.Module):
    def __init__(self, channels=3, features_d=64):
        """
        DCGAN Discriminator implementation
        
        Args:
            channels: Number of input channels (3 for RGB)
            features_d: Size factor for discriminator (higher = more capacity)
        """
        super(DCGANDiscriminator, self).__init__()
        
        self.main = nn.Sequential(
            # Input: 256x256x3
            nn.Conv2d(channels, features_d, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 128x128 -> 64x64
            nn.Conv2d(features_d, features_d * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 64x64 -> 32x32
            nn.Conv2d(features_d * 2, features_d * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 32x32 -> 16x16
            nn.Conv2d(features_d * 4, features_d * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 16x16 -> 8x8
            nn.Conv2d(features_d * 8, features_d * 16, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 16),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 8x8 -> 4x4
            nn.Conv2d(features_d * 16, features_d * 32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 32),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 4x4 -> 1x1
            nn.Conv2d(features_d * 32, 1, kernel_size=4, stride=1, padding=0, bias=False)
            # No sigmoid here - we'll use BCEWithLogitsLoss
        )
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)
    
    def forward(self, x):
        return self.main(x).view(-1, 1)

In [11]:
# For 256x256 images
class DCGAN256Generator(nn.Module):
    def __init__(self, z_dim=100, channels=3, features_g=64):
        super(DCGAN256Generator, self).__init__()
        self.z_dim = z_dim
        
        # Starting with 1x1 projection from noise
        self.project = nn.Linear(z_dim, features_g * 16 * 4 * 4)
        
        # Main convolutional blocks for upsampling
        self.main = nn.Sequential(
            # Block 1: 4x4 -> 8x8
            nn.ConvTranspose2d(features_g * 16, features_g * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),
            
            # Block 2: 8x8 -> 16x16
            nn.ConvTranspose2d(features_g * 8, features_g * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),
            
            # Block 3: 16x16 -> 32x32
            nn.ConvTranspose2d(features_g * 4, features_g * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),
            
            # Block 4: 32x32 -> 64x64
            nn.ConvTranspose2d(features_g * 2, features_g, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),
            
            # Block 5: 64x64 -> 128x128
            nn.ConvTranspose2d(features_g, features_g // 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g // 2),
            nn.ReLU(True),
            
            # Output layer: 128x128 -> 256x256
            nn.ConvTranspose2d(features_g // 2, channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()  # Output range: [-1, 1]
        )
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)
        elif classname.find('Linear') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias.data, 0)
    
    def forward(self, z):
        # Reshape noise to (batch_size, 1024, 4, 4)
        x = z.view(-1, 1024, 4, 4)  # Fixed channel count
        return self.main(x)


In [12]:

# For 256x256 images - Discriminator 
class DCGAN256Discriminator(nn.Module):
    def __init__(self, channels=3, features_d=64):
        super(DCGAN256Discriminator, self).__init__()
        
        self.main = nn.Sequential(
            # Input: 256x256x3
            nn.Conv2d(channels, features_d, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 128x128 -> 64x64
            nn.Conv2d(features_d, features_d * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 64x64 -> 32x32
            nn.Conv2d(features_d * 2, features_d * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 32x32 -> 16x16
            nn.Conv2d(features_d * 4, features_d * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 16x16 -> 8x8
            nn.Conv2d(features_d * 8, features_d * 16, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 16),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 8x8 -> 4x4
            nn.Conv2d(features_d * 16, features_d * 32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 32),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 4x4 -> 1x1
            nn.Conv2d(features_d * 32, 1, kernel_size=4, stride=1, padding=0, bias=False)
            # No sigmoid here - we'll use BCEWithLogitsLoss
        )
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)
    
    def forward(self, x):
        return self.main(x).view(-1, 1)

In [13]:
# For 256x256 STARE dataset
class DCGAN256STARE(nn.Module):
    def __init__(self, z_dim=100, channels=3, features_g=64):
        super(DCGAN256STARE, self).__init__()
        self.z_dim = z_dim
        self.features_g=features_g
        # Starting with projection from noise
        self.project = nn.Sequential(
            nn.Linear(z_dim, features_g * 16 * 4 * 4),
            nn.BatchNorm1d(features_g * 16 * 4 * 4),
            nn.ReLU(True)
        )
        
        # Main convolutional blocks for upsampling to 256x256
        self.main = nn.Sequential(
            # Block 1: 4x4 -> 8x8
            nn.ConvTranspose2d(features_g * 16, features_g * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),
            
            # Block 2: 8x8 -> 16x16
            nn.ConvTranspose2d(features_g * 8, features_g * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),
            
            # Block 3: 16x16 -> 32x32
            nn.ConvTranspose2d(features_g * 4, features_g * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),
            
            # Block 4: 32x32 -> 64x64
            nn.ConvTranspose2d(features_g * 2, features_g, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),
            
            # Block 5: 64x64 -> 128x128
            nn.ConvTranspose2d(features_g, features_g // 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g // 2),
            nn.ReLU(True),
            
            # Output layer: 128x128 -> 256x256
            nn.ConvTranspose2d(features_g // 2, channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()  # Output range: [-1, 1]
        )
        
        # Initialize weights using DCGAN paper recommendation
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)
        elif classname.find('Linear') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias.data, 0)
    
    def forward(self, z):
        x = self.project(z)   # Now this will work!
        x = x.view(-1, 1024, 4, 4)
        return self.main(x)

In [14]:
# For 256x256 STARE dataset - Discriminator with spectral normalization
class STAREDiscriminator(nn.Module):
    def __init__(self, channels=3, features_d=64, use_spectral_norm=True):
        super(STAREDiscriminator, self).__init__()
        
        # Choose between regular Conv and Spectral Normalized Conv
        if use_spectral_norm:
            conv_layer = lambda in_c, out_c, k, s, p: nn.utils.spectral_norm(
                nn.Conv2d(in_c, out_c, kernel_size=k, stride=s, padding=p, bias=False)
            )
        else:
            conv_layer = lambda in_c, out_c, k, s, p: nn.Conv2d(
                in_c, out_c, kernel_size=k, stride=s, padding=p, bias=False
            )
        
        self.main = nn.Sequential(
            # Input: 256x256x3
            conv_layer(channels, features_d, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 128x128 -> 64x64
            conv_layer(features_d, features_d * 2, 4, 2, 1),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 64x64 -> 32x32
            conv_layer(features_d * 2, features_d * 4, 4, 2, 1),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 32x32 -> 16x16
            conv_layer(features_d * 4, features_d * 8, 4, 2, 1),
            nn.BatchNorm2d(features_d * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 16x16 -> 8x8
            conv_layer(features_d * 8, features_d * 16, 4, 2, 1),
            nn.BatchNorm2d(features_d * 16),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 8x8 -> 4x4
            conv_layer(features_d * 16, features_d * 16, 4, 2, 1),
            nn.BatchNorm2d(features_d * 16),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 4x4 -> 1x1
            conv_layer(features_d * 16, 1, 4, 1, 0)
        )
        
        # Initialize weights if not using spectral norm
        if not use_spectral_norm:
            self.apply(self._init_weights)
        
    def _init_weights(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)
    
    def forward(self, x):
        return self.main(x).view(-1, 1)

In [15]:
# Example of how to use these models:
def create_models(z_dim=100, image_channels=3, features_g=64, features_d=64):
    """
    Create DCGAN models for a 256x256 STARE dataset
    
    Args:
        z_dim: Size of noise vector
        image_channels: Number of channels in the image (3 for RGB)
        features_g: Base feature size for generator
        features_d: Base feature size for discriminator
        
    Returns:
        generator, discriminator models
    """
    generator = DCGAN256STARE(z_dim=z_dim, channels=image_channels, features_g=features_g)
    discriminator = STAREDiscriminator(channels=image_channels, features_d=features_d, use_spectral_norm=True)
    
    return generator, discriminator

In [16]:
def generate_and_save_images(model, epoch, z_dim=100, num_images=16):
    model.eval()
    with torch.no_grad():
        z = torch.randn(num_images, z_dim, device=device)
        fake_images = model(z).detach().cpu()
        grid = vutils.make_grid(fake_images, nrow=4, normalize=True)
        plt.figure(figsize=(8, 8))
        plt.imshow(np.transpose(grid, (1, 2, 0)))
        plt.axis("off")
        plt.title(f"Generated Images - Epoch {epoch}")
        plt.show()
    model.train()

In [17]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
from lpips import LPIPS
import numpy as np
from tqdm import tqdm

# Create directories to save generated images
os.makedirs("generated_images", exist_ok=True)
os.makedirs("real_images", exist_ok=True)

def train_dcgan(G, D, dataloader, z_dim=100, epochs=100, device="cuda", 
                lr=0.0002, betas=(0.5, 0.999), save_interval=1):
    """
    Training function for DCGAN on STARE dataset
    
    Args:
        G: Generator model
        D: Discriminator model
        dataloader: DataLoader with STARE images
        z_dim: Dimensionality of noise vector
        epochs: Number of training epochs
        device: Device to train on ('cuda' or 'cpu')
        lr: Learning rate
        betas: Adam optimizer betas
        save_interval: How often to save images and evaluate metrics
    """
    # Loss function - using BCE with logits for numerical stability
    criterion = nn.BCEWithLogitsLoss()
    
    # Label smoothing - use 0.9 for real labels instead of 1.0
    real_label_val = 0.9
    fake_label_val = 0.0
    
    # Optimizers with proper hyperparameters for DCGAN
    optimizer_G = optim.Adam(G.parameters(), lr=lr, betas=betas)
    optimizer_D = optim.Adam(D.parameters(), lr=lr, betas=betas)
    
    # Create a dictionary to store metrics
    metrics = {
        'epochs': [],
        'fid': [],
        'kid': [],
        'lpips': [],
        'ssim': [],
        'psnr': [],
        'diversity': [],
        'd_losses': [],
        'g_losses': []
    }
    
    # Fixed noise for visualization
    torch.manual_seed(42)
    fixed_noise = torch.randn(64, z_dim, device=device)
    
    # Loss tracking
    D_losses = []
    G_losses = []
    
    # Initialize lpips model for evaluation
    lpips_model = LPIPS(net='alex').to(device).eval()
    
    # Training loop
    for epoch in range(epochs):
        epoch_d_loss = 0.0
        epoch_g_loss = 0.0
        
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for batch_idx, real_images in enumerate(progress_bar):
            # Get batch size (might be smaller for last batch)
            batch_size = real_images.size(0)
            real_images = real_images.to(device)
            
            # ===== Train Discriminator =====
            # Zero gradients
            optimizer_D.zero_grad()
            
            # Generate labels with noise for label smoothing
            real_labels = torch.full((batch_size, 1), real_label_val, device=device)
            # Add some noise to labels for stabilization
            real_labels = real_labels - 0.05 * torch.rand(real_labels.size(), device=device)
            
            # Forward pass real batch through D
            outputs_real = D(real_images)
            
            # Calculate loss on real images
            loss_real = criterion(outputs_real, real_labels)
            
            # Generate fake images batch
            noise = torch.randn(batch_size, z_dim, device=device)
            fake_images = G(noise)
            
            # Classify fake images
            fake_labels = torch.full((batch_size, 1), fake_label_val, device=device)
            outputs_fake = D(fake_images.detach())  # Detach to avoid training G
            
            # Calculate loss on fake images
            loss_fake = criterion(outputs_fake, fake_labels)
            
            # Combined D loss and backprop
            loss_D = loss_real + loss_fake
            loss_D.backward()
            optimizer_D.step()
            
            # ===== Train Generator =====
            # Zero G gradients
            optimizer_G.zero_grad()
            
            # Generate another batch of fake images (some implementations reuse, but fresh batch can help)
            noise = torch.randn(batch_size, z_dim, device=device)
            fake_images = G(noise)
            
            # Try to fool the discriminator
            outputs = D(fake_images)
            
            # G wants D to think these are real
            loss_G = criterion(outputs, real_labels)
            
            # Backprop and update G
            loss_G.backward()
            optimizer_G.step()
            
            # Update statistics
            epoch_d_loss += loss_D.item()
            epoch_g_loss += loss_G.item()
            
            # Update the progress bar
            progress_bar.set_postfix({
                'D_loss': f"{loss_D.item():.4f}", 
                'G_loss': f"{loss_G.item():.4f}"
            })
        
        # Calculate average epoch losses
        avg_d_loss = epoch_d_loss / len(dataloader)
        avg_g_loss = epoch_g_loss / len(dataloader)
        D_losses.append(avg_d_loss)
        G_losses.append(avg_g_loss)
        
        print(f"Epoch {epoch+1}/{epochs} | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}")
        
        # Generate and save images every save_interval epochs
        if epoch % save_interval == 0 or epoch == epochs - 1:
            # Generate fake images for evaluation
            G.eval()
            with torch.no_grad():
                fake_samples = G(fixed_noise).detach().cpu()
            G.train()
            
            # Adjust image count to prevent ValueError on smaller batches
            real_batch_size = min(64, real_images.size(0))
            real_images_vis = real_images[:real_batch_size].detach().cpu()
            fake_samples_vis = fake_samples[:real_batch_size]
            
            # Save generated & real images
            vutils.save_image(fake_samples_vis, f"generated_images/epoch_{epoch+1:03d}.png", 
                             normalize=True, nrow=8)
            vutils.save_image(real_images_vis, f"real_images/epoch_{epoch+1:03d}.png", 
                             normalize=True, nrow=8)
            
            # Evaluate metrics and update tracking
            fid_score, kid_score, lpips_avg, ssim_avg, psnr_avg, diversity_avg = evaluate_metrics(
                real_images_vis, fake_samples_vis, epoch, device)
            
            # Record metrics
            update_metrics(metrics, epoch, fid_score, kid_score, lpips_avg, ssim_avg, 
                          psnr_avg, diversity_avg, avg_d_loss, avg_g_loss)
            
            # Save model checkpoints
            if (epoch + 1) % 5 == 0:
                torch.save(G.state_dict(), f"model_checkpoints/generator_{epoch+1}.pth")

    
    return G, D, metrics

In [18]:
def update_metrics(metrics_dict, epoch, fid, kid, lpips, ssim, psnr, diversity, d_loss, g_loss):
    """Update metrics dictionary with values from current epoch"""
    metrics_dict['epochs'].append(epoch + 1)  # 1-indexed for plotting
    metrics_dict['fid'].append(fid)
    metrics_dict['kid'].append(kid)
    metrics_dict['lpips'].append(lpips)
    metrics_dict['ssim'].append(ssim)
    metrics_dict['psnr'].append(psnr)
    metrics_dict['diversity'].append(diversity)
    metrics_dict['d_losses'].append(d_loss)
    metrics_dict['g_losses'].append(g_loss)

In [19]:
def evaluate_metrics(real_images, fake_images, epoch, device):
    """
    Evaluate image quality metrics comparing real and generated images
    
    Args:
        real_images: Batch of real images
        fake_images: Batch of generated images
        epoch: Current epoch number
        device: Device to perform calculations on
        
    Returns:
        Tuple of (fid_score, kid_score, lpips_avg, ssim_avg, psnr_avg, diversity_avg)
    """
    from torchvision.models import inception_v3
    from torchvision.transforms import Compose, Resize, ToTensor, Normalize
    from skimage.metrics import structural_similarity as ssim
    from skimage.metrics import peak_signal_noise_ratio as psnr
    from torchmetrics.image.kid import KernelInceptionDistance
    import numpy as np
    from scipy import linalg
    import torchvision.transforms as transforms

    lpips_model = LPIPS(net='alex').to(device).eval()
    inception = inception_v3(pretrained=True, transform_input=False).to(device).eval()
    resize_norm = Compose([
        Resize((299, 299)),
        ToTensor(),
        Normalize([0.5]*3, [0.5]*3)
    ])

    # --- FID ---
    def get_activations(imgs):
        # Convert tensor images to PIL for transform
        if torch.is_tensor(imgs[0]):
            imgs = [transforms.ToPILImage()(img) for img in imgs]
            
        imgs = torch.stack([resize_norm(i) for i in imgs]).to(device)
        with torch.no_grad():
            # Get features before the final classification layer
            inception.aux_logits = False
            features = inception(imgs)
        return features.cpu().numpy()

    # Convert image formats for metrics calculations
    real_np = real_images.permute(0, 2, 3, 1).cpu().numpy()
    fake_np = fake_images.permute(0, 2, 3, 1).cpu().numpy()

    # Calculate Inception features
    act_real = get_activations([transforms.ToPILImage()(img) for img in real_images])
    act_fake = get_activations([transforms.ToPILImage()(img) for img in fake_images])

    # Calculate FID
    mu1, sigma1 = act_real.mean(axis=0), np.cov(act_real, rowvar=False)
    mu2, sigma2 = act_fake.mean(axis=0), np.cov(act_fake, rowvar=False)
    diff = mu1 - mu2
    
    # Calculate sqrt of product between cov
    covmean, _ = linalg.sqrtm(sigma1 @ sigma2, disp=False)
    if np.iscomplexobj(covmean): 
        covmean = covmean.real
    
    fid_score = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2 * covmean)

    # --- KID ---
    # Use smaller subset size if fewer images available
    subset_size = min(len(real_images), len(fake_images), 10)
    kid_metric = KernelInceptionDistance(subset_size=subset_size).to(device)
    
    # Convert images to uint8 format for KID
    real_uint8 = (real_images * 255).to(torch.uint8).to(device)
    fake_uint8 = (fake_images * 255).to(torch.uint8).to(device)
    
    # Update KID metric with images
    kid_metric.update(real_uint8, real=True)
    kid_metric.update(fake_uint8, real=False)
    
    # Compute KID
    kid_mean, kid_std = kid_metric.compute()
    kid_score = kid_mean.item()

    # --- LPIPS & Diversity ---
    lpips_scores = []
    diversity_scores = []
    num_eval = min(len(real_images), len(fake_images), 10)
    imgs = fake_images[:num_eval]
    
    for i in range(len(imgs)):
        for j in range(i+1, len(imgs)):
            # Scale images to [-1, 1] for LPIPS
            f1 = (imgs[i].unsqueeze(0).to(device) - 0.5) * 2
            f2 = (imgs[j].unsqueeze(0).to(device) - 0.5) * 2
            score = lpips_model(f1, f2).item()
            diversity_scores.append(score)
    
    for real, fake in zip(real_images[:num_eval], fake_images[:num_eval]):
        # Scale images to [-1, 1] for LPIPS
        r = (real.unsqueeze(0).to(device) - 0.5) * 2
        f = (fake.unsqueeze(0).to(device) - 0.5) * 2
        lpips_scores.append(lpips_model(r, f).item())

    # --- SSIM & PSNR ---
    ssim_scores = []
    psnr_scores = []
    for r, f in zip(real_np[:num_eval], fake_np[:num_eval]):
        r_img = (r * 255).astype(np.uint8)
        f_img = (f * 255).astype(np.uint8)
        ssim_scores.append(ssim(r_img, f_img, channel_axis=-1))
        psnr_scores.append(psnr(r_img, f_img))

    # Averages
    lpips_avg = np.mean(lpips_scores)
    ssim_avg = np.mean(ssim_scores)
    psnr_avg = np.mean(psnr_scores)
    diversity_avg = np.mean(diversity_scores) if diversity_scores else 0.0

    print(f"\n📊 Epoch {epoch + 1} Metrics:")
    print(f"✅ FID:       {fid_score:.4f}")
    print(f"✅ KID:       {kid_score:.4f}")
    print(f"✅ LPIPS:     {lpips_avg:.4f}")
    print(f"✅ SSIM:      {ssim_avg:.4f}")
    print(f"✅ PSNR:      {psnr_avg:.4f}")
    print(f"✅ Diversity: {diversity_avg:.4f}")

    return fid_score, kid_score, lpips_avg, ssim_avg, psnr_avg, diversity_avg

In [20]:
def plot_training_metrics(metrics):
    """Plot training metrics over time"""
    import matplotlib.pyplot as plt
    
    # Create directory for plots
    os.makedirs("training_plots", exist_ok=True)
    
    # Plot FID over epochs
    plt.figure(figsize=(10, 6))
    plt.plot(metrics['epochs'], metrics['fid'], marker='o', color='green')
    plt.title('FID Score Over Training')
    plt.xlabel('Epoch')
    plt.ylabel('FID Score (lower is better)')
    plt.grid(True)
    plt.savefig('training_plots/fid.png')
    plt.close()
    
    # Plot KID over epochs
    plt.figure(figsize=(10, 6))
    plt.plot(metrics['epochs'], metrics['kid'], marker='o', color='green')
    plt.title('KID Score Over Training')
    plt.xlabel('Epoch')
    plt.ylabel('KID Score (lower is better)')
    plt.grid(True)
    plt.savefig('training_plots/kid.png')
    plt.close()
    
    # Plot LPIPS over epochs
    plt.figure(figsize=(10, 6))
    plt.plot(metrics['epochs'], metrics['lpips'], marker='o', color='green')
    plt.title('LPIPS Score Over Training')
    plt.xlabel('Epoch')
    plt.ylabel('LPIPS (lower is better)')
    plt.grid(True)
    plt.savefig('training_plots/lpips.png')
    plt.close()
    
    # Plot losses
    plt.figure(figsize=(10, 6))
    plt.plot(metrics['epochs'], metrics['d_losses'], label='Discriminator Loss')
    plt.plot(metrics['epochs'], metrics['g_losses'], label='Generator Loss')
    plt.title('Training Losses')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig('training_plots/losses.png')
    plt.close()
    
    # Plot diversity
    plt.figure(figsize=(10, 6))
    plt.plot(metrics['epochs'], metrics['diversity'], marker='o', color='green')
    plt.title('Image Diversity Over Training')
    plt.xlabel('Epoch')
    plt.ylabel('Diversity Score (higher is better)')
    plt.grid(True)
    plt.savefig('training_plots/diversity.png')
    plt.close()

In [21]:
import torch.nn as nn
z_dim = 100
epochs = 100
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    
# Get models
G, D = create_models(z_dim=z_dim)
G.to(device)
D.to(device)    
# Create directory for model checkpoints
os.makedirs("model_checkpoints", exist_ok=True)
G_trained, D_trained, metrics = train_dcgan(G, D, dataloader, z_dim, epochs, device)    
# Plot metrics after training


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 198MB/s] 
/usr/local/lib/python3.10/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle mo

Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


Epoch 1/100: 100%|██████████| 224/224 [01:22<00:00,  2.70it/s, D_loss=0.6145, G_loss=3.2197] 


Epoch 1/100 | D Loss: 0.7703 | G Loss: 6.0847
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


...

Epoch 98/100 | D Loss: 0.4703 | G Loss: 5.5979
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth

📊 Epoch 98 Metrics:
✅ FID:       1161.1420
✅ KID:       0.0908
✅ LPIPS:     0.3913
✅ SSIM:      0.2503
✅ PSNR:      10.6613
✅ Diversity: 0.3412


Epoch 99/100: 100%|██████████| 224/224 [01:29<00:00,  2.51it/s, D_loss=0.3934, G_loss=3.9716] 


Epoch 99/100 | D Loss: 0.4240 | G Loss: 5.7051
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth

📊 Epoch 99 Metrics:
✅ FID:       1064.1102
✅ KID:       0.0128
✅ LPIPS:     0.4758
✅ SSIM:      0.2281
✅ PSNR:      10.2272
✅ Diversity: 0.5160


Epoch 100/100: 100%|██████████| 224/224 [01:29<00:00,  2.50it/s, D_loss=0.3789, G_loss=5.9963] 


Epoch 100/100 | D Loss: 0.4248 | G Loss: 5.8377
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth

📊 Epoch 100 Metrics:
✅ FID:       956.6345
✅ KID:       0.0544
✅ LPIPS:     0.3491
✅ SSIM:      0.2785
✅ PSNR:      10.6165
✅ Diversity: 0.3577


In [22]:
plot_training_metrics(metrics)

In [23]:
"""import shutil
import os

working_dir = "/kaggle/working"

# Loop through everything in the directory
for filename in os.listdir(working_dir):
    file_path = os.path.join(working_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # remove file or symbolic link
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # remove directory
    except Exception as e:
        print(f"Failed to delete {file_path}. Reason: {e}")

print("✅ All files and folders in /kaggle/working deleted.")"""

'import shutil\nimport os\n\nworking_dir = "/kaggle/working"\n\n# Loop through everything in the directory\nfor filename in os.listdir(working_dir):\n    file_path = os.path.join(working_dir, filename)\n    try:\n        if os.path.isfile(file_path) or os.path.islink(file_path):\n            os.unlink(file_path)  # remove file or symbolic link\n        elif os.path.isdir(file_path):\n            shutil.rmtree(file_path)  # remove directory\n    except Exception as e:\n        print(f"Failed to delete {file_path}. Reason: {e}")\n\nprint("✅ All files and folders in /kaggle/working deleted.")'